***Step-1 Importing Libraries and API Integeration***

***Step-2 Data Extracting and Coverting DataFrame for Category, Competition, Complexes, Venues, Competitor and Competitor Ranking***

In [1]:
import requests
import json
import pandas as pd
import sqlite3

# Step 1: Fetching Data from the Competitions API
url_competitions = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=xRFvyVa4aCxkKtS0938LmgHfWGhCR7J500ggKWmS"
headers = {"accept": "application/json"}
response_competitions = requests.get(url_competitions, headers=headers)
if response_competitions.status_code == 200:
    data_competitions = response_competitions.json()
else:
    print("Failed to fetch competitions data")
    data_competitions = {}

# Parse JSON and Extract Relevant Data for Competitions
competitions = data_competitions.get('competitions', [])
flat_data_competitions = []
for competition in competitions:
    flat_data_competitions.append({
        'id': competition.get('id'),
        'name': competition.get('name'),
        'gender': competition.get('gender'),
        'type': competition.get('type'),
        'category_id': competition.get('category', {}).get('id'),
        'category_name': competition.get('category', {}).get('name')
    })

# Converting to DataFrames
competitions_df = pd.DataFrame(flat_data_competitions)

# Step 2: Fetching Data from the Complexes API
url_complexes = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=xRFvyVa4aCxkKtS0938LmgHfWGhCR7J500ggKWmS"
response_complexes = requests.get(url_complexes, headers=headers)
if response_complexes.status_code == 200:
    data_complexes = response_complexes.json()
else:
    print("Failed to fetch complexes data")
    data_complexes = {}

# Parse JSON and Extract Relevant Data for Complexes
complexes = data_complexes.get('complexes', [])
flat_data_complexes = []
for complex_ in complexes:
    for venue in complex_.get('venues', []):
        flat_data_complexes.append({
            'complex_id': complex_.get('id'),
            'complex_name': complex_.get('name'),
            'venue_id': venue.get('id'),
            'venue_name': venue.get('name'),
            'city_name': venue.get('city'),
            'country_name': venue.get('country'),
            'country_code': venue.get('country_code'),
            'timezone': venue.get('timezone')
        })

# Converting to DataFrames
complexes_df = pd.DataFrame(flat_data_complexes)
venues_df = pd.DataFrame(flat_data_complexes)

# Step 3: Fetching Data from the Competitor Rankings API
url_rankings = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=xRFvyVa4aCxkKtS0938LmgHfWGhCR7J500ggKWmS"
response_rankings = requests.get(url_rankings, headers=headers)
if response_rankings.status_code == 200:
    data_rankings = response_rankings.json()
else:
    print("Failed to fetch rankings data")
    data_rankings = {}

# Extract and Transform Relevant Data for Competitor Rankings
rankings = data_rankings.get('rankings', [])
flat_data_rankings = []
for ranking in rankings:
    for competitor_ranking in ranking.get('competitor_rankings', []):
        competitor = competitor_ranking.get('competitor', {})
        flat_data_rankings.append({
            'type_id': ranking.get('type_id'),
            'name': ranking.get('name'),
            'year': ranking.get('year'),
            'week': ranking.get('week'),
            'gender': ranking.get('gender'),
            'rank': competitor_ranking.get('rank'),
            'movement': competitor_ranking.get('movement'),
            'points': competitor_ranking.get('points'),
            'competitions_played': competitor_ranking.get('competitions_played'),
            'competitor_id': competitor.get('id'),
            'competitor_name': competitor.get('name'),
            'country_name': competitor.get('country'),
            'country_code': competitor.get('country_code'),
            'abbreviation': competitor.get('abbreviation')
        })

# Converting to DataFrames
competitors_df = pd.DataFrame(flat_data_rankings)
competitor_rankings_df = pd.DataFrame(flat_data_rankings)

# Step 4: Store Data in a Relational Database
# Create a SQLite database 
db_file_path = r"D:\MTDM37\MTDM37\data\sports.db"
conn = sqlite3.connect(db_file_path)

# Drop existing tables if they exist
tables = ['Competitions', 'Complexes', 'Venues', 'Competitors', 'Competitor_Rankings']
cursor = conn.cursor()
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
conn.commit()

# Use if_exists='replace' to overwrite existing tables
competitions_df.to_sql('Competitions', conn, if_exists='replace', index=False)
complexes_df.to_sql('Complexes', conn, if_exists='replace', index=False)
venues_df.to_sql('Venues', conn, if_exists='replace', index=False)
competitors_df.to_sql('Competitors', conn, if_exists='replace', index=False)
competitor_rankings_df.to_sql('Competitor_Rankings', conn, if_exists='replace', index=False)

# Verify the data was stored correctly
print("Data stored in the database:")
print(pd.read_sql_query("SELECT * FROM Competitions", conn))
print(pd.read_sql_query("SELECT * FROM Complexes", conn))
print(pd.read_sql_query("SELECT * FROM Venues", conn))
print(pd.read_sql_query("SELECT * FROM Competitors", conn))
print(pd.read_sql_query("SELECT * FROM Competitor_Rankings", conn))

# Close the connection
conn.close()


Data stored in the database:
                        id                               name gender     type  \
0       sr:competition:620                         Hopman Cup  mixed    mixed   
1       sr:competition:660                     World Team Cup    men    mixed   
2       sr:competition:990         ATP Challenger Tour Finals    men  singles   
3      sr:competition:1207  Championship International Series  women  singles   
4      sr:competition:2100                          Davis Cup    men    mixed   
...                    ...                                ...    ...      ...   
5877  sr:competition:45913                  UTR Bucuresti W01  women  singles   
5878  sr:competition:45915              UTR Rungsted Kyst W01  women  singles   
5879  sr:competition:45917                  UTR Bucuresti W01  women  singles   
5880  sr:competition:45935      ATP Brussels, Belgium Singles    men  singles   
5881  sr:competition:45937      ATP Brussels, Belgium Doubles    men  doubles   

***Step-3 Store Data in a Relational Database***

In [2]:
import sqlite3
import pandas as pd

# Create a SQLite database 
db_file_path = r"D:\MTDM37\MTDM37\data\sports.db"
conn = sqlite3.connect(db_file_path)

# Drop existing tables if they exist
tables = ['Competitions', 'Complexes', 'Venues', 'Competitors', 'Competitor_Rankings']
cursor = conn.cursor()
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
conn.commit()

# Use if_exists='replace' to overwrite existing tables
competitions_df.to_sql('Competitions', conn, if_exists='replace', index=False)
complexes_df.to_sql('Complexes', conn, if_exists='replace', index=False)
venues_df.to_sql('Venues', conn, if_exists='replace', index=False)
competitors_df.to_sql('Competitors', conn, if_exists='replace', index=False)
competitor_rankings_df.to_sql('Competitor_Rankings', conn, if_exists='replace', index=False)

# Verify the data was stored correctly
print("Data stored in the database:")
print(pd.read_sql_query("SELECT * FROM Competitions", conn))
print(pd.read_sql_query("SELECT * FROM Complexes", conn))
print(pd.read_sql_query("SELECT * FROM Venues", conn))
print(pd.read_sql_query("SELECT * FROM Competitors", conn))
print(pd.read_sql_query("SELECT * FROM Competitor_Rankings", conn))

# Close the connection
conn.close()


Data stored in the database:
                        id                               name gender     type  \
0       sr:competition:620                         Hopman Cup  mixed    mixed   
1       sr:competition:660                     World Team Cup    men    mixed   
2       sr:competition:990         ATP Challenger Tour Finals    men  singles   
3      sr:competition:1207  Championship International Series  women  singles   
4      sr:competition:2100                          Davis Cup    men    mixed   
...                    ...                                ...    ...      ...   
5877  sr:competition:45913                  UTR Bucuresti W01  women  singles   
5878  sr:competition:45915              UTR Rungsted Kyst W01  women  singles   
5879  sr:competition:45917                  UTR Bucuresti W01  women  singles   
5880  sr:competition:45935      ATP Brussels, Belgium Singles    men  singles   
5881  sr:competition:45937      ATP Brussels, Belgium Doubles    men  doubles   

***Save each DataFrame to a CSV file***

In [3]:
import pandas as pd
import sqlite3
import os

# Connect to SQLite database and read the data into DataFrames
db_file_path = r"D:\MTDM37\MTDM37\data\sports.db"
conn = sqlite3.connect(db_file_path)

categories_df = pd.read_sql_query("SELECT * FROM Categories", conn)
competitions_df = pd.read_sql_query("SELECT * FROM competitions", conn)
complexes_df = pd.read_sql_query("SELECT * FROM complexes", conn)
venues_df = pd.read_sql_query("SELECT * FROM venues", conn)
competitors_df = pd.read_sql_query("SELECT * FROM Competitors", conn)
competitor_rankings_df = pd.read_sql_query("SELECT * FROM Competitor_Rankings", conn)

# Define the directory where CSV files will be saved
output_dir = r"D:\MTDM37\MTDM37\notebooks\Extracted CSV File"

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Save DataFrames to CSV files in the specified directory
categories_df.to_csv(os.path.join(output_dir, 'categories.csv'), index=False)
competitions_df.to_csv(os.path.join(output_dir, 'competitions.csv'), index=False)
complexes_df.to_csv(os.path.join(output_dir, 'complexes.csv'), index=False)
venues_df.to_csv(os.path.join(output_dir, 'venues.csv'), index=False)
competitors_df.to_csv(os.path.join(output_dir, 'competitors.csv'), index=False)
competitor_rankings_df.to_csv(os.path.join(output_dir, 'competitor_rankings.csv'), index=False)

# Close the connection
conn.close()

print("CSV files created successfully in the specified folder.")


CSV files created successfully in the specified folder.


***Read the CSV file and the database table into DataFrames and Compare the two DataFrames***

In [4]:
import pandas as pd
import sqlite3

# Function to check if a CSV file is already saved in the database
def check_csv_in_database(csv_file, table_name):
    # Read the CSV file into a DataFrame
    csv_df = pd.read_csv(csv_file)

    # Connect to SQLite database
    conn = sqlite3.connect('D:\\MTDM37\\MTDM37\\data\\sports.db')

    # Read the corresponding table from the database into a DataFrame
    db_df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Close the connection
    conn.close()

    # Check if the DataFrames are identical
    return csv_df.equals(db_df)

# List of CSV files and corresponding table names
csv_files = [
    'D:\\MTDM37\\MTDM37\\notebooks\\Extracted CSV File\\categories.csv', 
    'D:\\MTDM37\\MTDM37\\notebooks\\Extracted CSV File\\competitions.csv', 
    'D:\\MTDM37\\MTDM37\\notebooks\\Extracted CSV File\\complexes.csv', 
    'D:\\MTDM37\\MTDM37\\notebooks\\Extracted CSV File\\venues.csv', 
    'D:\\MTDM37\\MTDM37\\notebooks\\Extracted CSV File\\competitors.csv', 
    'D:\\MTDM37\\MTDM37\\notebooks\\Extracted CSV File\\competitor_rankings.csv'
]
table_names = ['Categories', 'Competitions', 'Complexes', 'Venues', 'Competitors', 'Competitor_Rankings']

# Check each CSV file against the corresponding table in the database
for csv_file, table_name in zip(csv_files, table_names):
    is_saved = check_csv_in_database(csv_file, table_name)
    if is_saved:
        print(f"The CSV file '{csv_file}' has already been saved in the database.")
    else:
        print(f"The CSV file '{csv_file}' has NOT been saved in the database.")


The CSV file 'D:\MTDM37\MTDM37\notebooks\Extracted CSV File\categories.csv' has already been saved in the database.
The CSV file 'D:\MTDM37\MTDM37\notebooks\Extracted CSV File\competitions.csv' has already been saved in the database.
The CSV file 'D:\MTDM37\MTDM37\notebooks\Extracted CSV File\complexes.csv' has NOT been saved in the database.
The CSV file 'D:\MTDM37\MTDM37\notebooks\Extracted CSV File\venues.csv' has NOT been saved in the database.
The CSV file 'D:\MTDM37\MTDM37\notebooks\Extracted CSV File\competitors.csv' has already been saved in the database.
The CSV file 'D:\MTDM37\MTDM37\notebooks\Extracted CSV File\competitor_rankings.csv' has already been saved in the database.


***Check if the Database File Exists***

In [5]:
import os

db_path = r"D:\MTDM37\MTDM37\data\sports.db"
if os.path.exists(db_path):
    print(f"The database file '{db_path}' exists.")
else:
    print(f"The database file '{db_path}' does NOT exist.")


The database file 'D:\MTDM37\MTDM37\data\sports.db' exists.


***Verify Table Creation***

In [6]:
import sqlite3
import pandas as pd

# Connect to SQLite database
db_file_path = r"D:\MTDM37\MTDM37\data\sports.db"
conn = sqlite3.connect(db_file_path)

# Query to list all tables
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)

print("Tables in the database:")
print(tables)

# List each table's columns
table_names = tables['name'].tolist()
for table in table_names:
    query = f"PRAGMA table_info({table});"
    columns = pd.read_sql_query(query, conn)
    print(f"\nColumns in '{table}' table:")
    print(columns)

# Close the connection
conn.close()


Tables in the database:
                  name
0           Categories
1         Competitions
2            Complexes
3               Venues
4          Competitors
5  Competitor_Rankings

Columns in 'Categories' table:
   cid           name  type  notnull dflt_value  pk
0    0             id  TEXT        0       None   0
1    1           name  TEXT        0       None   0
2    2         gender  TEXT        0       None   0
3    3           type  TEXT        0       None   0
4    4    category_id  TEXT        0       None   0
5    5  category_name  TEXT        0       None   0

Columns in 'Competitions' table:
   cid           name  type  notnull dflt_value  pk
0    0             id  TEXT        0       None   0
1    1           name  TEXT        0       None   0
2    2         gender  TEXT        0       None   0
3    3           type  TEXT        0       None   0
4    4    category_id  TEXT        0       None   0
5    5  category_name  TEXT        0       None   0

Columns in 'Complexe

***Verify Data in Each Table***

In [7]:
import sqlite3
import pandas as pd

# Connect to SQLite database
db_file_path = r"D:\MTDM37\MTDM37\data\sports.db"
conn = sqlite3.connect(db_file_path)

# Function to display sample data from each table
def display_table_sample(table_name):
    query = f"SELECT * FROM {table_name} LIMIT 5;"
    df = pd.read_sql_query(query, conn)
    print(f"\nSample data from '{table_name}' table:")
    print(df)

# List of table names to verify
tables = ['Categories', 'Competitions', 'Complexes', 'Venues', 'Competitors', 'Competitor_Rankings']

# Display sample data from each table
for table in tables:
    display_table_sample(table)

# Close the connection
conn.close()



Sample data from 'Categories' table:
                    id                               name gender     type  \
0   sr:competition:620                         Hopman Cup  mixed    mixed   
1   sr:competition:660                     World Team Cup    men    mixed   
2   sr:competition:990         ATP Challenger Tour Finals    men  singles   
3  sr:competition:1207  Championship International Series  women  singles   
4  sr:competition:2100                          Davis Cup    men    mixed   

       category_id category_name  
0  sr:category:181    Hopman Cup  
1    sr:category:3           ATP  
2   sr:category:72    Challenger  
3    sr:category:6           WTA  
4   sr:category:76     Davis Cup  

Sample data from 'Competitions' table:
                    id                               name gender     type  \
0   sr:competition:620                         Hopman Cup  mixed    mixed   
1   sr:competition:660                     World Team Cup    men    mixed   
2   sr:competition